In [1]:
#!/usr/bin/env python

# Load required modules and set constants
import sys
import os
import argparse, pandas as pd
import numpy as np

GENE = 'Hugo_Symbol'
SAMPLE = 'Tumor_Sample_Barcode'
VAR_TY = 'Variant_Type'
VAR_CLASS = 'Variant_Classification'
#PROTEIN_POS = 'Protein_position'
#AMINO_ACIDS = 'Amino_acids'
VAR_AA = 'Variant_amino_acid'
PROTEIN_CHANGE = 'Protein_Change'
AA_CHANGE = 'AA_Change'
VAR_AN = 'Variant_annotation'
VAR_AN2 = 'Variant annotation'

#DepMap_ID

#Usecols do not match columns, columns expected but not found: ['Variant_Type', 'Tumor_Sample_Barcode', 'Hugo_Symbol', 'Variant_Classification', 'Amino_acids', 'Protein_position', 'HGVSp_Short']

maf_col_dtype = {
    GENE: str,
    VAR_CLASS: str,
    VAR_TY: str,
    SAMPLE: str,
    PROTEIN_CHANGE: str
    #PROTEIN_POS: str,
    #AMINO_ACIDS: str,
}

ACTIVATING_CLASSES = { 'Missense_Mutation' }
#INACTIVATING_CLASSES = {'Nonsense_Mutation', 'Nonstop_Mutation',
                        #'Frame_Shift_Del', 'Frame_Shift_Ins',
                        #'Translation_Start_Site'}
INACTIVATING_CLASSES = {"Nonsense_Mutation", "Nonstop_Mutation", "Frame_Shift_Del",
                        "Frame_Shift_Ins", "Start_Codon_Del", "Stop_Codon_Del",
                        "Start_Codon_Ins", "Stop_Codon_Ins", "De_novo_Start_OutOfFrame", "Splice_Site"}

# Parse command-line arguments
#parser = argparse.ArgumentParser()
#parser.add_argument('-okf', '--oncoKB_file', type=str, required=True)
#parser.add_argument('-mf', '--maf_file', type=str, required=True)
#parser.add_argument('-sf', '--sample_file', type=str, required=True)
#parser.add_argument('-gf', '--gene_file', type=str, required=True)
#parser.add_argument('-of', '--output_file', type=str, required=True)
#args = parser.parse_args(sys.argv[1:])

oncoKB_file = "data/20Q2/raw/oncoKB_list.tsv"
sample_file = "data/20Q2/raw/sample_info.csv"
maf_file = "data/20Q2/raw/mutations.csv"
output_file = "data/20Q2/features/test_mutations_oncokb.csv"

# Load the sample and gene file
'''
with open(args.sample_file, 'r') as IN:
    arrs = [ l.rstrip('\n').split('\t') for l in IN if not l.startswith('#') ]
    samples = [ arr[1] for arr in arrs ]
    indices = [ int(arr[0]) for arr in arrs ]
    sample_index = dict(zip(samples, indices))
    sample_set = set(samples)

with open(args.gene_file, 'r') as IN:
    arrs = [ l.rstrip('\n').split('\t') for l in IN if not l.startswith('#') ]
    genes = [ arr[1] for arr in arrs ]
    indices = [ int(arr[0]) for arr in arrs ]
    gene_index = dict(zip(genes, indices))
    gene_set = set(genes)

print('- Loaded list of %s samples and %s genes' % (len(samples), len(genes)))
print("123")
'''



'\nwith open(args.sample_file, \'r\') as IN:\n    arrs = [ l.rstrip(\'\n\').split(\'\t\') for l in IN if not l.startswith(\'#\') ]\n    samples = [ arr[1] for arr in arrs ]\n    indices = [ int(arr[0]) for arr in arrs ]\n    sample_index = dict(zip(samples, indices))\n    sample_set = set(samples)\n\nwith open(args.gene_file, \'r\') as IN:\n    arrs = [ l.rstrip(\'\n\').split(\'\t\') for l in IN if not l.startswith(\'#\') ]\n    genes = [ arr[1] for arr in arrs ]\n    indices = [ int(arr[0]) for arr in arrs ]\n    gene_index = dict(zip(genes, indices))\n    gene_set = set(genes)\n\nprint(\'- Loaded list of %s samples and %s genes\' % (len(samples), len(genes)))\nprint("123")\n'

In [2]:
cwd = os.getcwd()
print(cwd)

/n/fs/ragr-research/projects/sd


In [4]:
# Load the processed oncoKB annotations
with open(oncoKB_file, 'r') as IN:
    arrs = [ l.rstrip('\n').split('\t') for l in IN if not l.startswith('Gene') ]
    genes = [ arr[0] for arr in arrs[0:] ]
    AAs = [ set(arr[1].split(',')) for arr in arrs[0:] ]
    geneclass = [ arr[2] for arr in arrs[0:] ]
    oncoKB_AAs = dict(zip(genes,AAs))
    oncoKB_geneclass = dict(zip(genes,geneclass))
    oncoKB_gene_set = set(genes)

print('- Loaded OncoKB annotations of %s genes' % (len(genes)))


FileNotFoundError: [Errno 2] No such file or directory: 'data/20Q2/raw/oncoKB_list.tsv'

In [5]:
# Load the MAF
with open(maf_file, 'r') as IN:
    #arrs = [ l.rstrip('\n').split('\t') for l in IN if not l.startswith('Gene') ]
    fl = IN.readline()
    header = fl.rstrip("\n").split("\t")
print(header)



print(maf_file)
maf = pd.read_table(maf_file, sep='\t', usecols=maf_col_dtype.keys(),dtype=maf_col_dtype)#,
                    #na_filter = False)
maf.fillna('NA',inplace=True)
maf[SAMPLE] = [ '-'.join(s.split('-')[:3]) for s in maf[SAMPLE] ]
#maf[VAR_AA] = [ aa.split('/')[1] if '/' in aa else aa for aa in maf[AMINO_ACIDS] ]
maf[AA_CHANGE] = maf.Protein_Change
maf[AA_CHANGE] = [ pc[2:] for pc in maf[AA_CHANGE] ]
#maf[AA_CHANGE] = [ pc[2:] for pc in maf[PROTEIN_CHANGE] if isinstance(pc,str) ]
maf[VAR_AN2] = ""
print('- Loaded a MAF with %s mutations in %s samples' % (len(maf), len(set(maf[SAMPLE]))))

FileNotFoundError: [Errno 2] No such file or directory: 'data/20Q2/raw/mutations.csv'

In [6]:
# Filter the MAF
maf = maf.loc[#maf[SAMPLE].isin(sample_set) & \
              #maf[GENE].isin(gene_set & oncoKB_gene_set) & \
              maf[GENE].isin(oncoKB_gene_set) & \
              #maf[VAR_TY].isin({'SNP'}) & \
              maf[VAR_CLASS].isin(ACTIVATING_CLASSES | INACTIVATING_CLASSES)]
#maf['Gene_Index'] = [ gene_index[g] for g in maf[GENE] ]
#maf['Sample_Index'] = [ sample_index[s] for s in maf[SAMPLE] ]
print('- Filtered to %s mutations in %s samples' % (len(maf), len(set(maf[SAMPLE]))))




NameError: name 'maf' is not defined

In [7]:
# Add corresponding OncoKB annotations for each gene
maf['AAs'] = maf.Hugo_Symbol.map(oncoKB_AAs)
maf['AAisinONCOKB'] = maf.apply(lambda r: r.AA_Change in r.AAs, axis=1)
maf['geneclass'] = maf.Hugo_Symbol.map(oncoKB_geneclass)



NameError: name 'maf' is not defined

In [8]:
# Classify Activating | Inactivating | Other
conditions = [(maf['geneclass'] == 'tumor_suppressor') & ((maf[VAR_CLASS].isin(INACTIVATING_CLASSES)) | maf['AAisinONCOKB']), # tsg: matching var_class or AA change
        (maf['geneclass'] == 'oncogene') & (maf[VAR_CLASS].isin(ACTIVATING_CLASSES)) & maf['AAisinONCOKB']] # oncogene: matching var_class and AA change

choices = ['inactivating', 'activating']
maf[VAR_AN2] = np.select(conditions, choices, default='other')
maf['Alteration'] = [ g+"_"+VA[0].upper()+"_MUT" for (g,VA) in zip(maf[GENE],maf[VAR_AN2])]



NameError: name 'maf' is not defined

In [9]:
mut_to_sample = dict()
for (g,s) in zip(maf['Alteration'],maf[SAMPLE]):
    if g not in mut_to_sample:
        mut_to_sample[g] = [s]
    else:
        if s not in mut_to_sample[g]:
            mut_to_sample[g].append(s)

toexclude = []
for k in mut_to_sample: 
    mut_to_sample[k] = len(mut_to_sample[k])
    if (mut_to_sample[k] < 10) and (k.endswith("O_MUT")):
        toexclude.append(k)
print(toexclude)

NameError: name 'maf' is not defined

In [10]:
maf2 = maf[~maf['Alteration'].isin(toexclude)]

NameError: name 'maf' is not defined

In [11]:
# Clean up dataframe
print(list(maf2.columns) )
maf3 = maf2[[GENE,SAMPLE,VAR_TY,VAR_CLASS,PROTEIN_CHANGE,AA_CHANGE,VAR_AN2,"Alteration"]]
print(list(maf3.columns) )
maf3.to_csv(output_file, sep=',', index=0)

NameError: name 'maf2' is not defined